<a href="https://colab.research.google.com/github/Vedant-Lotia-007/Fetch_ML_Task/blob/master/Text%20Summarization%20Using%20Transformers%20(BART).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install -q transformers datasets torch sentencepiece rouge-score accelerate
!pip install -q bitsandbytes  # Optional for memory-efficient training

import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
import numpy as np
import random


In [10]:
pip install wandb


In [13]:
# Ensure GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Check GPU details
if device == "cuda":
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("GPU Memory Allocated:", torch.cuda.memory_allocated())
    print("GPU Memory Reserved:", torch.cuda.memory_reserved())


Using device: cuda
GPU Name: Tesla T4
GPU Memory Allocated: 1625363456
GPU Memory Reserved: 1635778560


In [4]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"].select(range(20000))  # Select 20K samples for faster training
val_data = dataset["validation"].select(range(3000))
test_data = dataset["test"].select(range(3000))

print("Train Size:", len(train_data), "Validation Size:", len(val_data), "Test Size:", len(test_data))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Train Size: 20000 Validation Size: 3000 Test Size: 3000


In [5]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [6]:
max_input_length = 512
max_target_length = 150

def preprocess_data(batch):
    inputs = tokenizer(batch["article"], max_length=max_input_length, padding="max_length", truncation=True)
    targets = tokenizer(batch["highlights"], max_length=max_target_length, padding="max_length", truncation=True)

    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_data.map(preprocess_data, batched=True, remove_columns=["article", "highlights", "id"])
val_dataset = val_data.map(preprocess_data, batched=True, remove_columns=["article", "highlights", "id"])
test_dataset = test_data.map(preprocess_data, batched=True, remove_columns=["article", "highlights", "id"])


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = Seq2SeqTrainingArguments(
    output_dir="./fine_tuned_bart",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,  # Reduce learning rate to prevent overfitting
    per_device_train_batch_size=8,  # Optimize for T4 GPU
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Keeping it within 60 mins
    weight_decay=0.01,  # Prevent overfitting
    gradient_accumulation_steps=2,  # Reduce GPU memory usage
    fp16=True,  # Enable mixed precision training
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    predict_with_generate=True
)



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-18-17e23abe83c4>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.386000,0.653301
2,0.365300,0.678990
3,0.309400,0.716655


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3750, training_loss=0.35305943400065104, metrics={'train_runtime': 4816.3079, 'train_samples_per_second': 12.458, 'train_steps_per_second': 0.779, 'total_flos': 6.501313806336e+16, 'train_loss': 0.35305943400065104, 'epoch': 3.0})

In [29]:
from transformers import pipeline

# Create summarization pipeline using your fine-tuned model
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# Decode one example from val_dataset
example = val_dataset[0]

# Decode input and label
input_text = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
target_summary = tokenizer.decode(example["labels"], skip_special_tokens=True)

# Generate prediction
predicted_summary = summarizer(input_text, max_length=128, min_length=30, do_sample=False)[0]['summary_text']

print("🔹 Original Article:\n", input_text)
print("\n🔹 Target Summary:\n", target_summary)
print("\n🔹 Predicted Summary:\n", predicted_summary)


Device set to use cuda:0


🔹 Original Article:
 (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I'm just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard's gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pa

In [35]:
import shutil
from google.colab import files

# Create a directory to save model
output_dir = "./fine_tuned_bart_model"

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



('./fine_tuned_bart_model/tokenizer_config.json',
 './fine_tuned_bart_model/special_tokens_map.json',
 './fine_tuned_bart_model/vocab.json',
 './fine_tuned_bart_model/merges.txt',
 './fine_tuned_bart_model/added_tokens.json')